# Movie Galaxy — Semantic Movie Exploration


## Imports

In [ ]:
import os
import pandas as pd
import numpy as np

## Loading Dataset

In [ ]:
import kagglehub
movies_path = kagglehub.dataset_download("alanvourch/tmdb-movies-daily-updates")

In [ ]:
os.listdir(movies_path)

In [ ]:
movies_df = pd.read_csv(f"{movies_path}/TMDB_all_movies.csv")

## Exploring Dataset

In [ ]:
# download samble data to view in excel
# sample_movies = movies_df.sample(n=100, random_state=42)
# sample_movies.to_csv("movies_sample_100.csv", index=False)

In [ ]:
print("Shape:", movies_df.shape)
print("\nColumns:", movies_df.columns.tolist())
print("\nFirst 3 rows:")
display(movies_df.head(3))
print("\nInfo:")
movies_df.info()
print("\nMissing values (%):")
print((movies_df.isnull().sum() / len(movies_df) * 100).round(2).sort_values(ascending=False))
print("\nBasic stats:")
display(movies_df.describe())

In [ ]:
# title, overview, genres

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import ast

# Set plot style
sns.set_style("whitegrid")

# Plot 1: Number of movies released each year
print("\n--- Movies Released Each Year ---")
movies_df['release_year'] = pd.to_datetime(movies_df['release_date'], errors='coerce').dt.year
year_counts = movies_df['release_year'].value_counts().sort_index()

plt.figure(figsize=(15, 7))
sns.lineplot(x=year_counts.index, y=year_counts.values)
plt.title('Number of Movies Released Each Year')
plt.xlabel('Year')
plt.ylabel('Number of Movies')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Plot 2: Movies from each original language
# This plot tells most of the movies are in english so ignore this feature.
print("\n--- Movies by Original Language ---")
language_counts = movies_df['original_language'].value_counts().head(20) # Top 20 languages

plt.figure(figsize=(12, 7))
sns.barplot(x=language_counts.index, y=language_counts.values, hue=language_counts.index, palette='viridis', legend=False)
plt.title('Top 20 Original Languages of Movies')
plt.xlabel('Original Language')
plt.ylabel('Number of Movies')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Number of peoples voted
# for 66% of data the count is 0
zero_vote_count = movies_df[movies_df['vote_count'] == 0]
print(f"Number of movies with 0 vote count: {len(zero_vote_count)}")

total_movies = movies_df.shape[0]
percentage_zero_votes = (len(zero_vote_count) / total_movies) * 100
print(f"Percentage of movies with 0 vote count: {percentage_zero_votes:.2f}%")

In [ ]:
movies_df.shape

## Clean Data

In [ ]:
df = movies_df[movies_df['vote_count'] >= 30].copy()
df = df.sort_values('popularity', ascending=False).head(5000).copy()
df.shape

In [ ]:
print("Missing values for specified columns in df:")
print(df[['title', 'genres', 'overview']].isnull().sum())

In [ ]:
df['genres'] = df['genres'].fillna('Unknown')
df['cast'] = df['cast'].fillna('')
df['director'] = df['director'].fillna('Unknown')

df['year'] = df['release_year'].fillna(0).astype(int)
df = df.drop(columns=['release_year'])


In [ ]:
df = df[df['overview'].notna() & df['genres'].notna()].copy()

print("Shape after dropping rows with null 'overview' or 'genres':", df.shape)
print(df[['title', 'genres', 'overview']].isnull().sum())

In [ ]:
def build_natural_text(row):
    parts = []

    title = str(row['title']).strip()
    year = f" ({int(row['year'])})" if 'year' in row and pd.notna(row['year']) and row['year'] != 0 else ""
    parts.append(f"{title}{year}.")

    overview = str(row.get('overview', '')).strip()
    if overview:
        if not overview.endswith(('.', '!', '?')):
            overview += "."
        parts.append(overview)

    genres = str(row.get('genres', '')).strip()
    if genres and genres.lower() not in ['unknown', 'none', '']:
        parts.append(f"Genres: {genres}.")

    text = " ".join(parts).strip()
    return f"search_document: {text}"

# Uncomment to use this version instead
df['text_for_embedding'] = df.apply(build_natural_text, axis=1)

In [ ]:
for i in range(3):
    txt = df['text_for_embedding'].iloc[i]
    print(f"\nRow {i} ({df['title'].iloc[i]}):")
    print(txt[:600] + "..." if len(txt) > 600 else txt)

print("\nAverage text length:", df['text_for_embedding'].str.len().mean().round(0))
print("Any empty texts?", (df['text_for_embedding'].str.strip() == '').sum())

### Create Embeddings

In [ ]:
!pip install -q sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import torch
from tqdm.auto import tqdm

model_name = 'nomic-ai/nomic-embed-text-v1.5'

print(f"Loading model: {model_name}")
model = SentenceTransformer(model_name, trust_remote_code=True)

# Use GPU if available (Colab → change runtime to GPU)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)
print(f"Using device: {device}")

# Your texts
texts = df['text_for_embedding'].tolist()

print(f"Encoding {len(texts)} texts...")
embeddings = model.encode(
    texts,
    batch_size=128,
    show_progress_bar=True,
    normalize_embeddings=True,      # very important for cosine similarity
    convert_to_numpy=True
)

print("\nEmbeddings created!")
print("Shape:", embeddings.shape)      # should be (5000, 768)

# Save immediately
np.save('embeddings_5k_nomic.npy', embeddings)
print("Embeddings saved → embeddings_5k_nomic.npy")

### Visualisation

In [ ]:
import numpy as np
import pandas as pd
import os

# Embeddings are already in the 'embeddings' variable from the previous cell

# Prepare metadata
# Ensure 'df' is the filtered DataFrame used for embeddings
# It should contain 'title', 'genres', 'cast_top5', 'director', 'year'
# Check if 'cast_top5' and 'director' exist, otherwise use placeholders
metadata_cols = ['title', 'genres', 'year']
if 'cast_top5' in df.columns:
    metadata_cols.append('cast_top5')
else:
    df['cast_top5'] = '' # Add empty column if not present
if 'director' in df.columns:
    metadata_cols.append('director')
else:
    df['director'] = 'Unknown' # Add 'Unknown' if not present

metadata_df = df[metadata_cols].copy()

# Create a single label for each movie by concatenating relevant metadata
# This will be displayed when hovering over points in Projector
def create_label(row):
    parts = [f"{row['title']} ({int(row['year'])})"]
    if row['genres'] and row['genres'] != 'Unknown':
        parts.append(f"Genres: {row['genres']}")
    if 'cast_top5' in row and row['cast_top5']:
        parts.append(f"Cast: {row['cast_top5']}")
    if 'director' in row and row['director'] != 'Unknown':
        parts.append(f"Director: {row['director']}")
    return ' - '.join(parts)

metadata_df['label'] = metadata_df.apply(create_label, axis=1)

# Save embeddings to TSV
embeddings_tsv_path = 'embeddings.tsv'
np.savetxt(embeddings_tsv_path, embeddings, delimiter='\t')
print(f"Embeddings saved to {embeddings_tsv_path}")

# Save metadata to TSV
metadata_tsv_path = 'metadata.tsv'
# TensorBoard Projector expects metadata file without header and tab-separated
metadata_df['label'].to_csv(metadata_tsv_path, sep='\t', index=False, header=False)
print(f"Metadata saved to {metadata_tsv_path}")

print("\nNow we can upload these two files to https://projector.tensorflow.org/")

In [ ]:
!zip embeddings.zip embeddings.tsv metadata.tsv
from google.colab import files
files.download('embeddings.zip')

In [ ]:
from sentence_transformers.util import cos_sim

# Change indices to test different pairs
i, j = 0, 15

sim = cos_sim(embeddings[i], embeddings[j])[0][0].item()
print(f"Similarity between {df['title'].iloc[i]} and {df['title'].iloc[j]}: {sim:.4f}")

## FAISS search

In [ ]:
!pip install faiss-cpu

In [ ]:
import numpy as np
import faiss
import pandas as pd

# Load your data
# Use the embeddings array already in memory from the previous step (2DhlTt5Bk-4y)
# If you saved a specific 'embeddings_5k_nomic.npy', you could load it here:
embeddings = np.load('embeddings_5k_nomic.npy').astype('float32')

# Assuming 'df' is the cleaned and processed DataFrame already in memory.
# Remove the line that tries to load 'movies_5k_with_text.parquet' to avoid inconsistencies
# df = pd.read_parquet('movies_5k_with_text.parquet') # This line is removed

print("Embeddings shape:", embeddings.shape)
print("DataFrame shape:", df.shape)

In [ ]:
# Build FAISS Index (very fast for similarity search)
dim = embeddings.shape[1]   # 768

index = faiss.IndexFlatIP(dim)        # Inner Product = Cosine when normalized
index.add(embeddings)

faiss.write_index(index, 'faiss_index_5k.faiss')

print("FAISS index built and saved!")

In [ ]:
def find_similar_movies(query_title, top_k=10):
    # Find the row(s) in df where the title matches (case-insensitive)
    matching_rows_df = df[df['title'].str.contains(query_title, case=False, na=False)]

    if matching_rows_df.empty:
        print("Movie not found!")
        return

    # Get the *original index value* of the first matching movie
    movie_original_index_value = matching_rows_df.index[0]

    # Now, find the *positional integer index* (iloc) of this movie within the *current* df
    # This positional index will correctly map to the embeddings array
    movie_iloc = df.index.get_loc(movie_original_index_value)

    # Search using the correct positional embedding
    query_vector = embeddings[movie_iloc].reshape(1, -1)
    distances, indices = index.search(query_vector, top_k + 1)

    print(f"\n🎥 Movies similar to: **{df['title'].iloc[movie_iloc]}**\n")
    for rank, i in enumerate(indices[0][1:], 1):   # skip itself
        # 'i' here are the positional indices returned by FAISS, so we can use .iloc
        title = df['title'].iloc[i]
        print(f"{rank:2d}. {title} (Similarity: {distances[0][rank]:.4f})")

# Test it
find_similar_movies("Inception", top_k=8)
find_similar_movies("The Dark Knight", top_k=8)

## Create 3D Galaxy Visualization

In [ ]:
# Reduce to 3D using UMAP (best for galaxy look)
import umap
import plotly.express as px

print("Reducing to 3D... (this may take 30–90 seconds)")

reducer = umap.UMAP(
    n_components=3,
    n_neighbors=30,
    min_dist=0.1,
    random_state=42
)

umap_3d = reducer.fit_transform(embeddings)

# Add coordinates to dataframe
df['umap_x'] = umap_3d[:, 0]
df['umap_y'] = umap_3d[:, 1]
df['umap_z'] = umap_3d[:, 2]

print("3D coordinates created!")

In [ ]:
fig = px.scatter_3d(
    df,
    x='umap_x',
    y='umap_y',
    z='umap_z',
    hover_name='title',
    hover_data=['year', 'genres'],
    color='year',                    # color by year (beautiful gradient)
    # color='genres',                # or color by genre (if you want)
    title="🎥 Movie Galaxy - 3D Interactive (5,000 Movies)",
    opacity=0.8,
    width=1000,
    height=800
)

fig.update_traces(marker=dict(size=4))
fig.update_layout(scene=dict(
    xaxis_title='UMAP 1',
    yaxis_title='UMAP 2',
    zaxis_title='UMAP 3'
))

fig.show()

In [ ]:
fig.write_html("movie_galaxy_3d.html")
print("Saved as movie_galaxy_3d.html — open in browser!")

## Remsky dataset

In [1]:
!pip install -q -U datasets polars pyarrow faiss-cpu tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 12.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.4/810.4 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 MB 33.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 35.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 64.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 4.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.31.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-adk 1.21.0 requires google-cloud-bigquery-storage>=2.0.0, which is not installed.
dopamine-rl 4.1.2 requires gym<=0.25.2, but y

In [2]:
import polars as pl
from datasets import load_dataset
import numpy as np
import faiss
import os, gc
from tqdm.auto import tqdm

# folders
cache_dir = "/kaggle/working/hf_cache"
output_dir = "/kaggle/working/production_files"

os.makedirs(cache_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

print("Folders ready")

Folders ready


In [3]:
print("Downloading dataset to local Colab storage...")

ds = load_dataset(
    "Remsky/Embeddings__Ultimate_1Million_Movies_Dataset",
    split="train",
    cache_dir=cache_dir   # forces local storage
)

print("Download complete")
print("Total rows:", len(ds))

README.md: 0.00B [00:00, ?B/s]

movies_with_embeddings.parquet:   0%|          | 0.00/7.03G [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Download complete
Total rows: 1035695


In [4]:
print("Total columns:", len(ds.column_names))
print("\nColumns:\n")

for col in ds.column_names:
    print(col)

Total columns: 31

Columns:

id
title
vote_average
vote_count
status
release_date
revenue
runtime
budget
imdb_id
original_language
original_title
overview
popularity
tagline
genres
production_companies
production_countries
spoken_languages
movie_cast
director
director_of_photography
writers
producers
music_composer
imdb_rating
imdb_votes
poster_path
title_tagline_overview
token_count
embedding


In [5]:
import pprint
pprint.pprint(ds[2])

{'budget': 4000000.0,
 'director': 'Allison Anders, Alexandre Rockwell, Quentin Tarantino, Robert '
             'Rodriguez',
 'director_of_photography': 'Rodrigo García, Guillermo Navarro, Phil Parmet, '
                            'Andrzej Sekula',
 'embedding': '[-1.6385554,1.7313315,-4.1353016,0.09416334,-0.45942643,0.14296986,-0.84203047,0.46004412,-0.11383664,-0.23853554,-1.94238,-0.676626,-0.30694327,0.64624155,-0.1555678,-1.8658192,0.70476526,-0.3768293,-0.65744156,-0.7549398,-1.0417769,-0.4411411,-0.70778537,-0.7185255,0.7430849,0.51217103,-1.2560973,1.2714299,-0.29441094,-0.20855215,-0.24728215,0.52860075,-0.5995046,-1.0198188,-0.9413539,-0.43633723,0.4207148,0.37728694,-0.04276471,-0.39336705,0.36285686,0.25289592,-0.44944936,-0.2714941,0.10548811,-0.10478652,0.65955734,0.74743736,0.28412282,0.39682928,0.57749134,-0.72952265,0.5362237,0.4005261,1.3934237,0.47336182,-0.4519807,-0.06261601,0.5517217,1.1230259,1.2972453,-0.12460387,0.2385858,1.9681412,0.35841396,-0.42178053,0.2

In [6]:
KEEP_COLUMNS = [
    "id",
    "title",
    "release_date",
    "original_language",
    "overview",
    "genres",
    "movie_cast",
    "director",
    "writers",
    "runtime",
    "vote_average",
    "vote_count",
    "imdb_rating",
    "imdb_votes",
    "popularity",
    "poster_path",
    "embedding"
]
 
ds_clean = ds.remove_columns(
    [col for col in ds.column_names if col not in KEEP_COLUMNS]
)

print("Remaining columns:")
print(ds_clean.column_names)

Remaining columns:
['id', 'title', 'vote_average', 'vote_count', 'release_date', 'runtime', 'original_language', 'overview', 'popularity', 'genres', 'movie_cast', 'director', 'writers', 'imdb_rating', 'imdb_votes', 'poster_path', 'embedding']


In [7]:
from collections import defaultdict
from tqdm.auto import tqdm

null_counts = defaultdict(int)
total_rows = len(ds_clean)

batch_size = 5000

print("Calculating null percentages...")

for start in tqdm(range(0, total_rows, batch_size)):
    batch = ds_clean[start:start + batch_size]
    
    for col in ds_clean.column_names:
        values = batch[col]
        null_counts[col] += sum(v is None for v in values)

Calculating null percentages...


  0%|          | 0/208 [00:00<?, ?it/s]

In [8]:
print("\nNull Percentage per Column:\n")

null_percentages = {}

for col in ds_clean.column_names:
    pct = (null_counts[col] / total_rows) * 100
    null_percentages[col] = pct
    print(f"{col:25s} : {pct:.2f}%")


Null Percentage per Column:

id                        : 0.00%
title                     : 0.00%
vote_average              : 0.00%
vote_count                : 0.00%
release_date              : 11.19%
runtime                   : 0.00%
original_language         : 0.00%
overview                  : 17.69%
popularity                : 0.00%
genres                    : 28.93%
movie_cast                : 33.16%
director                  : 17.96%
writers                   : 51.41%
imdb_rating               : 58.11%
imdb_votes                : 58.11%
poster_path               : 28.97%
embedding                 : 0.00%


## Creating embeddings

In [9]:
import polars as pl
import numpy as np
import faiss
import os, gc
from tqdm.auto import tqdm

In [10]:
chunk_size = 6000

In [11]:
def parse_embedding(s):
    try:
        return np.fromstring(s[1:-1], sep=",", dtype=np.float32)
    except:
        return None

In [12]:
print("Generating production files...")

partial_meta_paths = []
partial_emb_paths = []

vector_offset = 0
chunk_idx = 0

for start in tqdm(range(0, len(ds_clean), chunk_size)):

    batch = ds_clean[start:start + chunk_size]
    df = pl.from_dicts(batch)

    # ---------- Keep full date ----------
    df = df.with_columns(
        pl.col("release_date")
        .str.strptime(pl.Date, "%Y-%m-%d", strict=False)
        .alias("release_date")
    )

    # ---------- Add YEAR ----------
    df = df.with_columns(
        pl.col("release_date")
        .dt.year()
        .fill_null(0)
        .cast(pl.Int32)
        .alias("year")
    )

    # ---------- Add vector_id ----------
    df = df.with_row_index("vector_id", offset=vector_offset) # this is what caused the 
    vector_offset += len(df)

    # ---------- Convert embeddings ----------
    df = df.with_columns(
        pl.col("embedding")
        .map_elements(parse_embedding, return_dtype=pl.Object)
        .alias("embedding_array")
    ).filter(pl.col("embedding_array").is_not_null())

    # ---------- Save metadata ----------
    meta = df.drop(["embedding", "embedding_array"])

    meta_path = f"{output_dir}/metadata_part_{chunk_idx:04d}.parquet"
    meta.write_parquet(meta_path)
    partial_meta_paths.append(meta_path)

    # ---------- Save embeddings ----------
    emb = np.vstack(df["embedding_array"].to_list())

    emb_path = f"{output_dir}/embeddings_part_{chunk_idx:04d}.npy"
    np.save(emb_path, emb)
    partial_emb_paths.append(emb_path)

    print(f"Chunk {chunk_idx} saved ({len(df)} rows)")
    chunk_idx += 1

    del df, meta, emb
    gc.collect()

print("Chunks finished.")

Generating production files...


  0%|          | 0/173 [00:00<?, ?it/s]

Chunk 0 saved (6000 rows)
Chunk 1 saved (6000 rows)
Chunk 2 saved (6000 rows)
Chunk 3 saved (6000 rows)
Chunk 4 saved (6000 rows)
Chunk 5 saved (6000 rows)
Chunk 6 saved (6000 rows)
Chunk 7 saved (6000 rows)
Chunk 8 saved (6000 rows)
Chunk 9 saved (6000 rows)
Chunk 10 saved (6000 rows)
Chunk 11 saved (6000 rows)
Chunk 12 saved (6000 rows)
Chunk 13 saved (6000 rows)
Chunk 14 saved (6000 rows)
Chunk 15 saved (6000 rows)
Chunk 16 saved (6000 rows)
Chunk 17 saved (6000 rows)
Chunk 18 saved (6000 rows)
Chunk 19 saved (6000 rows)
Chunk 20 saved (6000 rows)
Chunk 21 saved (6000 rows)
Chunk 22 saved (6000 rows)
Chunk 23 saved (6000 rows)
Chunk 24 saved (6000 rows)
Chunk 25 saved (6000 rows)
Chunk 26 saved (6000 rows)
Chunk 27 saved (6000 rows)
Chunk 28 saved (6000 rows)
Chunk 29 saved (6000 rows)
Chunk 30 saved (6000 rows)
Chunk 31 saved (6000 rows)
Chunk 32 saved (6000 rows)
Chunk 33 saved (6000 rows)
Chunk 34 saved (6000 rows)
Chunk 35 saved (6000 rows)
Chunk 36 saved (6000 rows)
Chunk 37 sa

In [13]:
metadata_final = pl.concat(
    (pl.read_parquet(p) for p in sorted(partial_meta_paths)),
    rechunk=True
)

metadata_final.write_parquet(f"{output_dir}/metadata.parquet")

print("Metadata shape:", metadata_final.shape)

Metadata shape: (1035695, 17)


In [14]:
emb_files = sorted([
    f"{output_dir}/{f}"
    for f in os.listdir(output_dir)
    if f.startswith("embeddings_part_")
])

sample = np.load(emb_files[0])
dim = sample.shape[1]

total_rows = sum(np.load(f).shape[0] for f in emb_files)

embeddings_path = f"{output_dir}/embeddings.npy"

embeddings_final = np.memmap(
    embeddings_path,
    dtype="float32",
    mode="w+",
    shape=(total_rows, dim)
)

cursor = 0
for f in tqdm(emb_files):
    chunk = np.load(f)
    n = chunk.shape[0]
    embeddings_final[cursor:cursor+n] = chunk
    cursor += n

embeddings_final.flush()

print("Embeddings ready:", embeddings_final.shape)

  0%|          | 0/173 [00:00<?, ?it/s]

Embeddings ready: (1035695, 768)


In [15]:
embeddings = np.memmap(
    f"{output_dir}/embeddings.npy",
    dtype="float32",
    mode="r",
    shape=(total_rows, dim)
)

index = faiss.IndexFlatIP(dim)
index.add(embeddings)

faiss.write_index(index, f"{output_dir}/faiss_index.faiss")

print("FAISS index saved ✓")

FAISS index saved ✓


In [16]:
os.listdir(output_dir)

['metadata_part_0006.parquet',
 'metadata_part_0158.parquet',
 'metadata_part_0132.parquet',
 'embeddings_part_0133.npy',
 'metadata_part_0106.parquet',
 'metadata_part_0000.parquet',
 'metadata_part_0084.parquet',
 'metadata_part_0080.parquet',
 'embeddings_part_0011.npy',
 'embeddings_part_0143.npy',
 'metadata_part_0168.parquet',
 'metadata_part_0112.parquet',
 'embeddings_part_0031.npy',
 'embeddings_part_0057.npy',
 'metadata_part_0128.parquet',
 'embeddings_part_0163.npy',
 'embeddings_part_0106.npy',
 'embeddings_part_0102.npy',
 'embeddings_part_0001.npy',
 'embeddings_part_0093.npy',
 'embeddings_part_0144.npy',
 'metadata_part_0159.parquet',
 'embeddings_part_0005.npy',
 'embeddings_part_0138.npy',
 'metadata_part_0157.parquet',
 'embeddings_part_0044.npy',
 'metadata_part_0035.parquet',
 'metadata_part_0153.parquet',
 'embeddings_part_0121.npy',
 'embeddings_part_0019.npy',
 'metadata_part_0014.parquet',
 'metadata_part_0149.parquet',
 'metadata_part_0067.parquet',
 'metadat

In [32]:
!rm -rf /kaggle/working/hf_cache
!rm -rf /root/.cache/huggingface
!rm -rf /root/.julia

In [40]:
!du -h --max-depth=1 / | sort -hr | head

du: cannot access '/proc/412/task/412/fd/3': No such file or directory
du: cannot access '/proc/412/task/412/fdinfo/3': No such file or directory
du: cannot access '/proc/412/fd/4': No such file or directory
du: cannot access '/proc/412/fdinfo/4': No such file or directory
32G	/
20G	/usr
9.5G	/kaggle
1.8G	/tools
536M	/root
136M	/var
55M	/content
51M	/datalab
21M	/python-apt
3.7M	/etc


In [34]:
!du -h --max-depth=1 /root | sort -hr

536M	/root
219M	/root/.npm
207M	/root/.cache
109M	/root/.EasyOCR
2.4M	/root/.launchpadlib
172K	/root/.ipython
96K	/root/.local
88K	/root/.jupyter
16K	/root/.config


In [28]:
!du -sh /kaggle/working/production_files

9.3G	/kaggle/working/production_files


In [37]:
import os

PRODUCTION_DIR = "/kaggle/working/production_files"

# files to keep
KEEP_FILES = {
    "metadata.parquet",
    "embeddings.npy",
    "faiss_index.faiss"
}

for fname in os.listdir(PRODUCTION_DIR):
    fpath = os.path.join(PRODUCTION_DIR, fname)

    if fname not in KEEP_FILES and os.path.isfile(fpath):
        os.remove(fpath)

print("✅ Cleanup complete")
print("Remaining files:")
print(os.listdir(PRODUCTION_DIR))

✅ Cleanup complete
Remaining files:
['faiss_index.faiss', 'metadata.parquet', 'embeddings.npy']


## Post production

In [38]:
import os
import shutil

# ---------- PATHS ----------
OUTPUT_DIR = "/kaggle/working/production_files"
FINAL_DIR = "/kaggle/working/final_export"

os.makedirs(FINAL_DIR, exist_ok=True)

print("Copying metadata and FAISS index...")

# Copy metadata
shutil.copy(
    f"{OUTPUT_DIR}/metadata.parquet",
    f"{FINAL_DIR}/metadata.parquet"
)

# Copy FAISS index
shutil.copy(
    f"{OUTPUT_DIR}/faiss_index.faiss",
    f"{FINAL_DIR}/faiss_index.faiss"
)

print("✅ Metadata and FAISS index copied")

print("\nCurrent final_export contents:")
print(os.listdir(FINAL_DIR))

Copying metadata and FAISS index...
✅ Metadata and FAISS index copied

Current final_export contents:
['faiss_index.faiss', 'metadata.parquet']


In [43]:
# !rm /kaggle/working/production_files/faiss_index.faiss
# !rm /kaggle/working/production_files/metadata.parquet
# !rm /kaggle/working/final_export/embeddings.npy

In [44]:
import numpy as np
import polars as pl
import os

PRODUCTION_DIR = "/kaggle/working/production_files"
FINAL_DIR = "/kaggle/working/final_export"

meta = pl.read_parquet(f"{FINAL_DIR}/metadata.parquet")

rows = meta.height
dim = 768

print("Opening memmap...")

src = np.memmap(
    f"{PRODUCTION_DIR}/embeddings.npy",
    dtype="float32",
    mode="r",
    shape=(rows, dim)
)

print("Creating destination array (streamed)...")

dst = np.lib.format.open_memmap(
    f"{FINAL_DIR}/embeddings.npy",
    mode="w+",
    dtype="float32",
    shape=(rows, dim)
)

# copy in chunks (VERY IMPORTANT)
chunk_size = 5000

for i in range(0, rows, chunk_size):
    dst[i:i+chunk_size] = src[i:i+chunk_size]

del dst
del src

print("✅ Conversion finished without storage spike")

Opening memmap...
Creating destination array (streamed)...
✅ Conversion finished without storage spike


In [46]:
# !rm /kaggle/working/production_files/embeddings.npy

In [45]:
# ---------- Verify ----------
emb = np.load(f"{FINAL_DIR}/embeddings.npy")

print("\nVerification:")
print("Shape:", emb.shape)
print("Dtype:", emb.dtype)

print("\nFinal folder contents:")
print(os.listdir(FINAL_DIR))


Verification:
Shape: (1035695, 768)
Dtype: float32

Final folder contents:
['faiss_index.faiss', 'metadata.parquet', 'embeddings.npy']


## Testing

In [47]:
import polars as pl
import numpy as np
import faiss

DATA_PATH = "/kaggle/working/final_export"

print("Loading files...")

metadata = pl.read_parquet(f"{DATA_PATH}/metadata.parquet")

embeddings = np.load(f"{DATA_PATH}/embeddings.npy")

index = faiss.read_index(f"{DATA_PATH}/faiss_index.faiss")

print("Metadata rows:", metadata.height)
print("Embedding shape:", embeddings.shape)
print("FAISS vectors:", index.ntotal)

Loading files...
Metadata rows: 1035695
Embedding shape: (1035695, 768)
FAISS vectors: 1035695


In [52]:
import polars as pl

DATA_PATH = "/kaggle/working/final_export"

# metadata = pl.read_parquet(f"{DATA_PATH}/metadata.parquet")

# add alignment index
metadata = metadata.with_row_index("vector_id")

# overwrite safely
metadata.write_parquet(f"{DATA_PATH}/metadata.parquet")

print("✅ vector_id added")
print(metadata.select(["vector_id", "title"]).head())

✅ vector_id added
shape: (5, 2)
┌───────────┬─────────────────────────────────┐
│ vector_id ┆ title                           │
│ ---       ┆ ---                             │
│ u32       ┆ str                             │
╞═══════════╪═════════════════════════════════╡
│ 0         ┆ Ariel                           │
│ 1         ┆ Shadows in Paradise             │
│ 2         ┆ Four Rooms                      │
│ 3         ┆ Judgment Night                  │
│ 4         ┆ Life in Loops (A Megacities RM… │
└───────────┴─────────────────────────────────┘


In [53]:
print(metadata.columns)

['vector_id', 'id', 'title', 'vote_average', 'vote_count', 'release_date', 'runtime', 'original_language', 'overview', 'popularity', 'genres', 'movie_cast', 'director', 'writers', 'imdb_rating', 'imdb_votes', 'poster_path', 'year']


In [85]:
title_to_id = {
    t.lower(): vid
    for t, vid in zip(
        metadata["title"],
        metadata["vector_id"]
    )
    if t is not None
}

def recommend(movie_title, k=10):

    movie_title = movie_title.lower()

    if movie_title not in title_to_id:
        print("❌ Movie not found")
        return

    vector_id = title_to_id[movie_title]

    query_vec = embeddings[vector_id].reshape(1, -1)

    distances, indices = index.search(query_vec, k + 1)

    results = (
        metadata[indices[0]]
        .select(["title", "year", "genres"])
        .with_columns(
            pl.Series("similarity", distances[0])
        )
    )

    # remove itself
    return results[1:]
recommend("Batman")

title,year,genres,similarity
str,i32,str,f32
"""Batman: Gotham Knight""",2008,"""Science Fiction, Animation, Ac…",15.472511
"""The Dark Knight""",2008,"""Drama, Action, Crime, Thriller""",15.229376
"""Batman Unmasked: The Psycholog…",2008,"""Documentary""",15.221379
"""Batman""",1989,"""Fantasy, Action, Crime""",14.88922
"""Batman Returns""",1992,"""Action, Fantasy""",14.775779
"""Batman: Mystery of the Batwoma…",2003,"""Adventure, Fantasy, Animation,…",14.759225
"""Batman: Mask of the Phantasm""",1993,"""Action, Animation, Crime, Myst…",14.739231
"""Batman & Robin""",1997,"""Action, Science Fiction, Adven…",14.651075
"""Protection""",2001,"""Drama, Action, Thriller""",14.520536


In [ ]:
# FESS vs Metadata alignment 
import faiss
import numpy as np
import polars as pl

meta = pl.read_parquet("/kaggle/working/final_export/metadata.parquet")
index = faiss.read_index("/kaggle/working/final_export/faiss_index.faiss")

print("Metadata rows:", meta.height)
print("FAISS vectors:", index.ntotal)

## Moving data to HF

In [57]:
!pip install -q huggingface_hub

In [58]:
from huggingface_hub import login
login()

In [60]:
from huggingface_hub import create_repo

create_repo(
    repo_id="abhijit26/movie-vector-galaxy",
    repo_type="dataset",
    exist_ok=True
)

RepoUrl('https://huggingface.co/datasets/abhijit26/movie-vector-galaxy', endpoint='https://huggingface.co', repo_type='dataset', repo_id='abhijit26/movie-vector-galaxy')

In [62]:
from huggingface_hub import upload_folder

upload_folder(
    folder_path="/kaggle/working/final_export",
    repo_id="abhijit26/movie-vector-galaxy",
    repo_type="dataset"
)

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/datasets/abhijit26/movie-vector-galaxy/commit/a43ea87c0a545072c6f5cfd447793c1c7e2027b8', commit_message='Upload folder using huggingface_hub', commit_description='', oid='a43ea87c0a545072c6f5cfd447793c1c7e2027b8', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/abhijit26/movie-vector-galaxy', endpoint='https://huggingface.co', repo_type='dataset', repo_id='abhijit26/movie-vector-galaxy'), pr_revision=None, pr_num=None)

In [ ]:
# I can create the ML based recommandation using some of those 17 features 